## Analyze cross validation results

In [1]:
import collections
import json
import numpy as np
import os
import pandas as pd
import yaml
import pprint
import scipy.stats
import tqdm

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [2]:
# Read yaml files
cross_val_dir = os.path.join(os.getenv("DATA_DIR"),
                             "mica_text_coref/movie_coref/results/coreference/cross_val_Dec06-07")
files_with_empty_results = []
rows = []
for directory in tqdm.tqdm(os.listdir(cross_val_dir), unit="dir"):
    result_file = os.path.join(cross_val_dir, directory, "result.yaml")
    with open(result_file, "r") as f:
        result = yaml.load(f, Loader=yaml.FullLoader)
    file_rows = []
    for document_len, document_len_metric in result["test_metric"].items():
        for overlap_len, overlap_len_metric in document_len_metric.items():
            for strategy, strategy_metric in overlap_len_metric.items():
                row = [result["preprocess"], result["bert_lr"], result["coref_lr"], result["warmup"],
                       document_len, overlap_len, strategy, result["test_movie"]]
                for data_metric in [result["dev_metric"], strategy_metric]:
                    metric_f1s = []
                    for metric in ["muc", "bcub", "ceafe"]:
                        for score in ["precision", "recall", "f1"]:
                            row.append(data_metric["span"][metric][score])
                        metric_f1s.append(data_metric["span"][metric]["f1"])
                    row.append(np.mean(metric_f1s))
                file_rows.append(row)
    if len(file_rows) == 0:
        files_with_empty_results.append(directory)
    rows += file_rows
data = np.array(rows)

hyperparams = ["preprocess", "bert_lr", "model_lr", "warmup", "document_len", "overlap_len", "strategy"]
score_cols = []
for dataset in ["dev", "test"]:
    for metric in ["muc", "bcub", "ceafe"]:
        for score in ["p", "r", "f"]:
            score_cols.append(f"{dataset}_{metric}_{score}")
    score_cols.append(f"{dataset}_conll_f1")
cross_val_df = pd.DataFrame(rows, columns=hyperparams + ["test_movie"] + score_cols)

print(f"shape = {cross_val_df.shape}")
display(cross_val_df)
print(f"directories with empty results: {files_with_empty_results}")
print()

print("Columns:")
for column_name, dtype in zip(cross_val_df.columns[:8], cross_val_df.dtypes[:8]):
    unique_vals = cross_val_df[column_name].unique().tolist()
    print(f"\t{column_name:20s} ({dtype}) : {unique_vals}")

100%|██████████| 720/720 [00:11<00:00, 61.25dir/s]

shape = (2142, 28)


,preprocess,bert_lr,model_lr,warmup,document_len,overlap_len,strategy,test_movie,dev_muc_p,dev_muc_r,dev_muc_f,dev_bcub_p,dev_bcub_r,dev_bcub_f,dev_ceafe_p,dev_ceafe_r,dev_ceafe_f,dev_conll_f1,test_muc_p,test_muc_r,test_muc_f,test_bcub_p,test_bcub_r,test_bcub_f,test_ceafe_p,test_ceafe_r,test_ceafe_f,test_conll_f1
0,addsays,0.00005,0.0002,0.0,20480,512,avg,zootopia,89.98,90.44,90.21,72.01,78.08,74.92,61.67,45.38,52.29,72.473333,93.20,91.27,92.23,79.95,64.08,71.14,45.46,33.39,38.50,67.290000
1,addsays,0.00005,0.0002,0.0,20480,1024,avg,zootopia,89.98,90.44,90.21,72.01,78.08,74.92,61.67,45.38,52.29,72.473333,93.02,91.30,92.15,79.56,64.19,71.06,44.44,34.21,38.66,67.290000
2,addsays,0.00005,0.0002,0.0,20480,2048,avg,zootopia,89.98,90.44,90.21,72.01,78.08,74.92,61.67,45.38,52.29,72.473333,93.15,91.27,92.20,76.97,63.21,69.42,47.42,35.67,40.71,67.443333
3,regular,0.00002,0.0001,0.5,10240,512,avg,john_wick,89.98,91.18,90.58,68.54,73.87,71.11,53.64,45.04,48.96,70.216667,94.03,92.93,93.48,82.81,74.27,78.31,37.11,70.94,48.73,73.506667
4,regular,0.00002,0.0001,0.5,10240,1024,avg,john_wick,89.98,91.18,90.58,68.54,73.87,71.11,53.64,45.04,48.96,70.216667,93.88,92.81,93.34,85.61,77.09,81.13,43.20,74.96,54.81,76.426667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,regular,0.00002,0.0001,-1.0,10240,1024,avg,prestige,90.50,90.74,90.62,80.18,76.47,78.28,61.26,46.81,53.07,73.990000,93.30,93.41,93.36,78.90,73.68,76.20,29.78,66.57,41.15,70.236667
2138,regular,0.00002,0.0001,-1.0,10240,2048,avg,prestige,90.50,90.74,90.62,80.18,76.47,78.28,61.26,46.81,53.07,73.990000,93.40,93.43,93.42,79.28,63.75,70.67,30.36,66.97,41.78,68.623333
2139,regular,0.00001,0.0002,-1.0,20480,512,avg,avengers_endgame,88.92,91.52,90.20,62.11,77.18,68.83,52.19,41.85,46.45,68.493333,92.26,92.91,92.58,48.44,58.08,52.82,29.97,42.21,35.05,60.150000
2140,regular,0.00001,0.0002,-1.0,20480,1024,avg,avengers_endgame,88.92,91.52,90.20,62.11,77.18,68.83,52.19,41.85,46.45,68.493333,92.29,92.83,92.56,44.18,59.34,50.65,29.64,37.58,33.14,58.783333


directories with empty results: ['Dec09_11:19:31PM_avengers_endgame', 'Dec09_09:36:06PM_avengers_endgame', 'Dec10_01:58:40AM_zootopia', 'Dec10_12:47:38AM_zootopia', 'Dec09_03:35:19PM_john_wick', 'Dec09_04:36:32PM_dead_poets_society']

Columns:
	preprocess           (object) : ['addsays', 'regular']
	bert_lr              (float64) : [5e-05, 2e-05, 1e-05]
	model_lr             (float64) : [0.0002, 0.0001]
	warmup               (float64) : [0.0, 0.5, 0.25, 1.0, -1.0]
	document_len         (int64) : [20480, 10240]
	overlap_len          (int64) : [512, 1024, 2048]
	strategy             (object) : ['avg']
	test_movie           (object) : ['zootopia', 'john_wick', 'quiet_place', 'avengers_endgame', 'dead_poets_society', 'prestige']


In [3]:
# Aggregate (macro) coreference performance scores for each movie
index_rows = []
data_rows = []
hyperparams = ["preprocess", "bert_lr", "model_lr", "warmup", "document_len", "overlap_len", "strategy"]
for hparam_values, df in cross_val_df.groupby(hyperparams):
    if len(df) == 6:
        row = df.iloc[:,len(hparam_values) + 1:].agg(["mean", "std"]).values.T.flatten().tolist()
        index_rows.append(list(hparam_values))
        data_rows.append(row)

index = pd.MultiIndex.from_product([["dev"], ["muc", "bcub", "ceafe"], ["p", "r", "f"], ["mean", "std"]],
                                   sortorder=None)
index = index.union(pd.MultiIndex.from_product([["dev"], ["conll"], ["f"], ["mean", "std"]], sortorder=None),
                    sort=False)
index = index.union(pd.MultiIndex.from_product([["test"], ["muc", "bcub", "ceafe"], ["p", "r", "f"], ["mean", "std"]],
                                               sortorder=None), sort=False)
index = index.union(pd.MultiIndex.from_product([["test"], ["conll"], ["f"], ["mean", "std"]], sortorder=None),
                    sort=False)
index.names = ["dataset", "metric", "score", "func"]

cross_val_index = pd.MultiIndex.from_frame(pd.DataFrame(index_rows, columns=hyperparams))
cross_val_agg_df = pd.DataFrame(data_rows, columns=index, index=cross_val_index)

display(cross_val_agg_df)
print(f"shape = {cross_val_agg_df.shape}")

dataset                                                                     dev  \
metric                                                                      muc   
score                                                                         p   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
addsays    0.00001 0.0001   -1.0   10240        512         avg       89.785000   
                                                1024        avg       89.785000   
                                                2048        avg       89.785000   
                                   20480        512         avg       89.785000   
                                                1024        avg       89.785000   
...                                                                         ...   
regular    0.00005 0.0002    1.0   10240        1024        avg       90.156667   
                                                2048        avg       90.156667   
                                   20480        512         avg       90.156667   
                                                1024        avg       90.156667   
                                                2048        avg       90.156667   

dataset                                                                         \
metric                                                                           
score                                                                            
func                                                                       std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy             
addsays    0.00001 0.0001   -1.0   10240        512         avg       0.557127   
                                                1024        avg       0.557127   
                                                2048        avg       0.557127   
                                   20480        512         avg       0.557127   
                                                1024        avg       0.557127   
...                                                                        ...   
regular    0.00005 0.0002    1.0   10240        1024        avg       2.260864   
                                                2048        avg       2.260864   
                                   20480        512         avg       2.260864   
                                                1024        avg       2.260864   
                                                2048        avg       2.260864   

dataset                                                                          \
metric                                                                            
score                                                                         r   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
addsays    0.00001 0.0001   -1.0   10240        512         avg       91.993333   
                                                1024        avg       91.993333   
                                                2048        avg       91.993333   
                                   20480        512         avg       91.993333   
                                                1024        avg       91.993333   
...                                                                         ...   
regular    0.00005 0.0002    1.0   10240        1024        avg       86.000000   
                                                2048        avg       86.000000   
                                   20480        512         avg       86.000000   
                                                1024        avg       86.000000   
                                                2048        avg       86.000000   

dataset                      

shape = (342, 40)


In [4]:
# Find hyperparameter configs for which results are empty
hyperparams_ext = hyperparams + ["test_movie"]
hyperparams_ext_listoftups = []
for hyperparam in hyperparams_ext:
    hparam_vals = sorted(cross_val_df[hyperparam].unique().tolist())
    hyperparams_ext_listoftups.append(hparam_vals)
allpermute_hyperparam_df = pd.MultiIndex.from_product(hyperparams_ext_listoftups).to_frame(index=False)
allpermute_hyperparam_df.columns = hyperparams_ext
allpermute_hyperparam_df = allpermute_hyperparam_df.sort_values(hyperparams_ext)

hyperparam_df = cross_val_df.loc[:, hyperparams_ext].copy()
indicator = allpermute_hyperparam_df.merge(hyperparam_df, how="left", indicator=True)["_merge"]
rem_df = allpermute_hyperparam_df[indicator == "left_only"]
display(rem_df)

,preprocess,bert_lr,model_lr,warmup,document_len,overlap_len,strategy,test_movie
794,addsays,0.00005,0.0001,0.25,10240,512,avg,john_wick
800,addsays,0.00005,0.0001,0.25,10240,1024,avg,john_wick
806,addsays,0.00005,0.0001,0.25,10240,2048,avg,john_wick
865,addsays,0.00005,0.0001,1.00,10240,512,avg,dead_poets_society
871,addsays,0.00005,0.0001,1.00,10240,1024,avg,dead_poets_society
877,addsays,0.00005,0.0001,1.00,10240,2048,avg,dead_poets_society
936,addsays,0.00005,0.0002,0.00,10240,512,avg,avengers_endgame
942,addsays,0.00005,0.0002,0.00,10240,1024,avg,avengers_endgame
948,addsays,0.00005,0.0002,0.00,10240,2048,avg,avengers_endgame
954,addsays,0.00005,0.0002,0.00,20480,512,avg,avengers_endgame


In [6]:
# maximum test set performance
cross_val_agg_df.loc[cross_val_agg_df["test"]["conll"]["f"]["mean"]
                     == cross_val_agg_df["test"]["conll"]["f"]["mean"].max(), ("test", slice(None), "f")]

dataset                                                                    test  \
metric                                                                      muc   
score                                                                         f   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
regular    0.00002 0.0001   -1.0   20480        2048        avg       93.621667   

dataset                                                                         \
metric                                                                           
score                                                                            
func                                                                       std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy             
regular    0.00002 0.0001   -1.0   20480        2048        avg       1.223461   

dataset                                                                     \
metric                                                                bcub   
score                                                                    f   
func                                                                  mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy         
regular    0.00002 0.0001   -1.0   20480        2048        avg       73.2   

dataset                                                                          \
metric                                                                            
score                                                                             
func                                                                        std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
regular    0.00002 0.0001   -1.0   20480        2048        avg       11.976626   

dataset                                                                          \
metric                                                                    ceafe   
score                                                                         f   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
regular    0.00002 0.0001   -1.0   20480        2048        avg       46.708333   

dataset                                                                         \
metric                                                                           
score                                                                            
func                                                                       std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy             
regular    0.00002 0.0001   -1.0   20480        2048        avg       8.389394   

dataset                                                                          \
metric                                                                    conll   
score                                                                         f   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
regular    0.00002 0.0001   -1.0   20480        2048        avg       71.176667   

dataset                                                                         
metric                                                                          
score                                                                           
func                                                                       std  
preprocess bert_lr model_lr warmup document_len overlap_len strategy            
regular    0.00002 0.0001   -1.0   20480        2048        avg       6.752167

In [64]:
# maximum dev set performance
cross_val_agg_df.loc[cross_val_agg_df["dev"]["conll"]["f"]["mean"]
                     == cross_val_agg_df["dev"]["conll"]["f"]["mean"].max(), ("dev", slice(None), "f")]

dataset                                                                     dev  \
metric                                                                      muc   
score                                                                         f   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
addsays    0.00005 0.0001   0.0    10240        512         avg       89.706667   
                                                1024        avg       89.706667   
                                                2048        avg       89.706667   
                                   20480        512         avg       89.706667   
                                                1024        avg       89.706667   
                                                2048        avg       89.706667   

dataset                                                                         \
metric                                                                           
score                                                                            
func                                                                       std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy             
addsays    0.00005 0.0001   0.0    10240        512         avg       0.855048   
                                                1024        avg       0.855048   
                                                2048        avg       0.855048   
                                   20480        512         avg       0.855048   
                                                1024        avg       0.855048   
                                                2048        avg       0.855048   

dataset                                                                          \
metric                                                                     bcub   
score                                                                         f   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
addsays    0.00005 0.0001   0.0    10240        512         avg       73.816667   
                                                1024        avg       73.816667   
                                                2048        avg       73.816667   
                                   20480        512         avg       73.816667   
                                                1024        avg       73.816667   
                                                2048        avg       73.816667   

dataset                                                                        \
metric                                                                          
score                                                                           
func                                                                      std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy            
addsays    0.00005 0.0001   0.0    10240        512         avg       2.36768   
                                                1024        avg       2.36768   
                                                2048        avg       2.36768   
                                   20480        512         avg       2.36768   
                                                1024        avg       2.36768   
                                                2048        avg       2.36768   

dataset                                                                      \
metric                                                                ceafe   
score                                                                     f   
func                                                                   mean   
preprocess bert_lr model_lr warmup document_len overlap_len s

In [65]:
# test set performance at maximum dev set performance
cross_val_agg_df.loc[cross_val_agg_df["dev"]["conll"]["f"]["mean"]
                     == cross_val_agg_df["dev"]["conll"]["f"]["mean"].max(), ("test", slice(None), "f")]

dataset                                                                    test  \
metric                                                                      muc   
score                                                                         f   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
addsays    0.00005 0.0001   0.0    10240        512         avg       92.963333   
                                                1024        avg       92.960000   
                                                2048        avg       93.003333   
                                   20480        512         avg       92.951667   
                                                1024        avg       92.990000   
                                                2048        avg       93.105000   

dataset                                                                         \
metric                                                                           
score                                                                            
func                                                                       std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy             
addsays    0.00005 0.0001   0.0    10240        512         avg       1.362317   
                                                1024        avg       1.394460   
                                                2048        avg       1.340308   
                                   20480        512         avg       1.310197   
                                                1024        avg       1.352938   
                                                2048        avg       1.239689   

dataset                                                                          \
metric                                                                     bcub   
score                                                                         f   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
addsays    0.00005 0.0001   0.0    10240        512         avg       66.251667   
                                                1024        avg       69.698333   
                                                2048        avg       70.468333   
                                   20480        512         avg       67.726667   
                                                1024        avg       69.096667   
                                                2048        avg       69.465000   

dataset                                                                          \
metric                                                                            
score                                                                             
func                                                                        std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
addsays    0.00005 0.0001   0.0    10240        512         avg       10.889822   
                                                1024        avg        9.989906   
                                                2048        avg       10.238925   
                                   20480        512         avg       11.718882   
                                                1024        avg       11.437835   
                                                2048        avg       11.016511   

dataset                                                                          \
metric                                                                    ceafe   
score                                                                         f   
func                                                                       mean   
preprocess bert_lr mode

In [62]:
# top 50 test set performances
display(cross_val_agg_df.sort_values(("test", "conll", "f", "mean"), ascending=False)
                        .loc[:,(slice(None), "conll", "f", "mean")]
                        .iloc[:50])

dataset                                                                     dev  \
metric                                                                    conll   
score                                                                         f   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
regular    0.00002 0.0001   -1.00  20480        2048        avg       69.984444   
                             0.25  20480        2048        avg       70.026667   
                            -1.00  20480        1024        avg       69.984444   
                                   10240        2048        avg       69.984444   
           0.00005 0.0001    0.25  20480        2048        avg       71.479444   
           0.00001 0.0002   -1.00  20480        2048        avg       69.022778   
           0.00002 0.0002    0.50  10240        2048        avg       71.279444   
           0.00001 0.0002    0.00  20480        2048        avg       71.243889   
           0.00002 0.0002    0.50  20480        2048        avg       71.279444   
                   0.0001    0.25  10240        2048        avg       70.026667   
addsays    0.00001 0.0002    0.25  20480        2048        avg       69.724444   
regular    0.00001 0.0002    0.00  20480        1024        avg       71.243889   
           0.00002 0.0001   -1.00  20480        512         avg       69.984444   
                             0.25  20480        1024        avg       70.026667   
           0.00001 0.0002   -1.00  10240        2048        avg       69.022778   
           0.00002 0.0002    0.50  20480        1024        avg       71.279444   
           0.00001 0.0002   -1.00  20480        1024        avg       69.022778   
           0.00002 0.0001    0.25  10240        1024        avg       70.026667   
                                   20480        512         avg       70.026667   
           0.00001 0.0001    1.00  20480        2048        avg       70.239444   
           0.00002 0.0002    0.50  10240        1024        avg       71.279444   
addsays    0.00001 0.0002    0.50  20480        1024        avg       69.426111   
                             0.00  10240        2048        avg       70.041111   
                             0.25  20480        1024        avg       69.724444   
regular    0.00001 0.0002    0.00  10240        1024        avg       71.243889   
addsays    0.00005 0.0002    1.00  20480        2048        avg       71.280556   
           0.00001 0.0002    0.50  20480        2048        avg       69.426111   
regular    0.00005 0.0001    0.25  20480        1024        avg       71.479444   
           0.00001 0.0001    0.50  20480        2048        avg       70.049444   
                             1.00  20480        1024        avg       70.239444   
           0.00002 0.0002   -1.00  20480        2048        avg       71.305556   
addsays    0.00001 0.0002    1.00  20480        2048        avg       70.079444   
regular    0.00002 0.0002   -1.00  10240        2048        avg       71.305556   
           0.00001 0.0002   -1.00  20480        512         avg       69.022778   
addsays    0.00001 0.0002    1.00  20480        1024        avg       70.079444   
regular    0.00005 0.0001    1.00  20480        2048        avg       69.903889   
addsays    0.00001 0.0002    0.00  20480        2048        avg       70.041111   
regular    0.00005 0.0002   -1.00  10240        2048        avg       69.673889   
                   0.0001    0.25  10240        2048        avg       71.479444   
           0.00002 0.0001    0.00  20480        2048        avg       71.017222   
           0.00005 0.0002   -1.00  20480        2048        avg       69.673889   
addsays    0.00001 0.0001    0.50  20480        2048        avg       68.971111   
regular    0.00001 0.0002    0.00  20480        512         avg       71.243889   
addsays    0.000

In [68]:
cross_val_df.groupby("test_movie").agg({"test_conll_f1": "max"}).mean()

test_conll_f1    73.558333
dtype: float64